In [ ]:
    *      Client              Server
   ORM     Python              (R)DBMS     ----      Databse
           sqlite3             SQLite                FILE
                   connect
                    cursor(SQL, Params)
                   <-----------> Business Logic != Dev
           => SNS(포스팅 등록, 수정)

In [93]:
import sqlite3

In [94]:
con = sqlite3.connect('sns.db')
cur = con.cursor()

In [95]:
#Table 3개
# Post             Hashtag           PostHashtag(N:M)
# PK, Content      PK, Tag, Cnt      P.PK, H.PK
cur.executescript('''
    DROP TABLE IF EXISTS POST;
    DROP TABLE IF EXISTS HASHTAG;
    DROP TABLE IF EXISTS PH;
    
    CREATE TABLE POST(
        PNO INTEGER PRIMARY KEY,
        CONTENT TEXT NOT NULL
    );

    CREATE TABLE HASHTAG(
        HNO INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL,
        CNT INTEGER NOT NULL DEFAULT 0
    );

    CREATE TABLE PH(
        PNO INTEGER NOT NULL,
        HNO INTEGER NOT NULL
    );
''')

In [96]:
# 새로운 게시물 등록 (내용, 해시태그들)
# 해시태그; (1)태그풀에 존재하는 태그만, (2)존재하지 않으면 새롭게 추가
def findTag(tag):
    cur.execute('SELECT HNO FROM HASHTAG WHERE NAME=?', (tag,))
    result = cur.fetchone() # fetchall => fetchone
    if not result: # result가 None 일때
        return 0
    else: # None이 아닐 때, fetchone이니깐 tuple(열, 열, ...)
        return result[0]

def addTag(tag):
    cur.execute('INSERT INTO HASHTAG(NAME) VALUES(?)', (tag,))
    con.commit()
    return cur.lastrowid
    
findTag('태그1')#, addTag('태그1')

0

In [97]:
def plusCount(hno):
    cur.execute('UPDATE HASHTAG SET CNT = CNT + 1 WHERE HNO=?', (hno,))
    con.commit()

def minusCount(hno):
    cur.execute('UPDATE HASHTAG SET CNT = CNT - 1 WHERE HNO=?', (hno,))
    con.commit()

In [98]:
plusCount(findTag('태그1')), plusCount(findTag('태그1')), plusCount(findTag('태그1'))

(None, None, None)

In [99]:
plusCount(findTag('태그2')), plusCount(findTag('태그3'))

(None, None)

In [100]:
def addPosting(content, opt=True, *tags):
    cur.execute('INSERT INTO POST(CONTENT) VALUES(?)', (content,))
    con.commit()
    pno = cur.lastrowid
    # if cur.lastrowid < 1:
    #     return None

    for tag in tags:
        # (1) 태그풀에 존재하는 애들만
        r = findTag(tag)
        # (2) 태그풀에 존재하지 않으면, 추가
        if r == 0 and opt == True:
            r = addTag(tag) # 새로운 태그가 들어가면, return pk
        if r > 0:
            plusCount(r) # 사용된 태그에 +1
            cur.execute('INSERT INTO PH VALUES(?,?)', (pno, r))
            con.commit()
            
    return pno

In [101]:
addPosting('내용1', False, '태그1', '태그2', '태그3')

1

In [102]:
addPosting('내용2', False, '태그1', '태그2', '태그3')

2

In [ ]:
Post => 2개
Hatshtag => 1개
PH => 2개

In [103]:
cur.execute('SELECT * FROM POST')
cur.fetchall()

[(1, '내용1'), (2, '내용2')]

In [110]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 1)]

In [111]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(3, 1), (3, 2), (3, 3)]

In [112]:
addPosting('내용3', True, '태그1', '태그2', '태그3')

4

In [113]:
cur.execute('SELECT * FROM POST')
cur.fetchall()

[(1, '내용1'), (2, '내용2'), (3, '내용3'), (4, '내용3')]

In [114]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 2), (2, '태그2', 2), (3, '태그3', 2)]

In [115]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(3, 1), (3, 2), (3, 3), (4, 1), (4, 2), (4, 3)]

In [63]:
def searchPostByTag(tag):
    hno = findTag(search(tag)[0])
    cur.execute('SELECT DISTINCT(CONTENT) FROM POST INNER JOIN PH ON PH.PNO=POST.PNO AND PH.HNO=?', (hno,))
    print(cur.fetchall())
searchPostByTag('태그1')

NameError: name 'search' is not defined

In [67]:
from sqlalchemy import create_engine, MetaData

engine = create_engine("sqlite:///your_database.db")  # Use your actual database URL
meta = MetaData()
meta.reflect(bind=engine)

In [69]:
meta.tables

FacadeDict({})

In [71]:
Table('USER', meta
      Column('PK', Integer, primary_key=True)
      COlumn('NAME', Text, nullable = False)
     )

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3019756352.py, line 1)

In [ ]:
meat.create_all(engine)

In [72]:
from sqlalchemy.sql import select, insert, join

In [77]:
print(select(), end='\n\n')
print(select(meta.tables['INFO']), end='\n\n')
print(select(meta.tables['INFO']).where(meta.tables['INFO'].c['PK']==1))

SELECT



KeyError: 'INFO'

In [78]:
user = meta.tables['USER']
info = meta.tables['INFO']

KeyError: 'USER'

In [ ]:
ORM: SQL X, Object 접근, 메모리 상에서 작업, DB와 작업환경을 분리

                Client                  Server
        ORM     Python     Sqlite       SqLite -------- Database(FILE)
            engine 객체를 생성
            DBMS 주소 전달 DBAPI://계정@주소:포트/데이터베이스
                                    X     X    => sqlite:///FILE경로      DBMS 접속 X (Lazy)
    Engine
Connect   Dialect
접속유지관리  객체 -> SQL로 해석
            engine.connect()
기존:                           execute      SQL
이제: Meta -> Table 객체 등록 (생성, reflect)
             [생성] Table 객체를 생성자로 만들어야 함(Params..., Column객체)
      Meta.tables[table 객체들이 들어있음]
             SQL 객체들로 호출(select, insert, ...)
            engine.connect().execute(객체를 전달) -> dialect -> SQL 변환 -> engine.connect -> [DBMS -> DB]
객체: Table, Column, SQL(select, insert, update, delete, join...)

In [79]:
dir(select())

['__annotations__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_add_context_option',
 '_all_selected_columns',
 '_annotate',
 '_annotations',
 '_annotations_cache_key',
 '_assert_no_memoizations',
 '_attributes',
 '_auto_correlate',
 '_cache_key_traversal',
 '_clone',
 '_clone_annotations_traverse_internals',
 '_cloned_set',
 '_compile_options',
 '_compile_state_factory',
 '_compile_state_plugin',
 '_compile_w_cache',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_correlate',
 '_correlate_except',


In [1]:
meta.clear()
con.close()

NameError: name 'meta' is not defined

In [4]:
engine = create_engine('sqlite:///coffee.db', echo = True)
con = engine.connect()

NameError: name 'create_engine' is not defined

In [3]:
meta.table.keys()

NameError: name 'meta' is not defined